In [1]:
import pandas as pd
from app.utils.data_loader import load_csv
from config import get_historical_files_path

In [2]:
file_path = get_historical_files_path('Bleached Softwood Kraft Pulp Futures Historical Data.csv')
data = load_csv(file_path)

In [3]:
def data_preprocessing(raw_data):
    data = raw_data.copy()

    data['Date'] = pd.to_datetime(data['Date'])
    data['Year'] = data['Date'].dt.year
    data['Month'] = data['Date'].dt.month
    data['Day'] = data['Date'].dt.day

    return data

In [4]:
data = data_preprocessing(data)

In [14]:
data

,Date,Price,Open,High,Low,Vol.,Change %,Year,Month,Day
0,2024-09-18,"5,564.0","5,580.0","5,600.0","5,510.0",1.15K,0.22%,2024,9,18
1,2024-09-13,"5,552.0","5,532.0","5,578.0","5,526.0",0.31K,0.29%,2024,9,13
2,2024-09-12,"5,536.0","5,566.0","5,582.0","5,510.0",0.18K,-0.18%,2024,9,12
3,2024-09-11,"5,546.0","5,602.0","5,624.0","5,502.0",2.18K,-1.42%,2024,9,11
4,2024-09-10,"5,626.0","5,616.0","5,670.0","5,612.0",0.36K,-0.04%,2024,9,10
...,...,...,...,...,...,...,...,...,...,...
1138,2020-01-08,"4,764.0","4,718.0","4,800.0","4,718.0",1.52K,1.28%,2020,1,8
1139,2020-01-07,"4,704.0","4,698.0","4,720.0","4,688.0",0.59K,-0.04%,2020,1,7
1140,2020-01-06,"4,706.0","4,722.0","4,734.0","4,688.0",0.74K,-0.72%,2020,1,6
1141,2020-01-03,"4,740.0","4,738.0","4,750.0","4,730.0",0.16K,-0.08%,2020,1,3


In [16]:
data[data['Year'] == 2024]

,Date,Price,Open,High,Low,Vol.,Change %,Year,Month,Day
0,2024-09-18,"5,564.0","5,580.0","5,600.0","5,510.0",1.15K,0.22%,2024,9,18
1,2024-09-13,"5,552.0","5,532.0","5,578.0","5,526.0",0.31K,0.29%,2024,9,13
2,2024-09-12,"5,536.0","5,566.0","5,582.0","5,510.0",0.18K,-0.18%,2024,9,12
3,2024-09-11,"5,546.0","5,602.0","5,624.0","5,502.0",2.18K,-1.42%,2024,9,11
4,2024-09-10,"5,626.0","5,616.0","5,670.0","5,612.0",0.36K,-0.04%,2024,9,10
...,...,...,...,...,...,...,...,...,...,...
168,2024-01-08,"5,854.0","5,856.0","5,932.0","5,790.0",524.72K,0.17%,2024,1,8
169,2024-01-05,"5,844.0","5,900.0","5,900.0","5,782.0",459.63K,-1.28%,2024,1,5
170,2024-01-04,"5,920.0","5,948.0","5,992.0","5,846.0",482.45K,0.54%,2024,1,4
171,2024-01-03,"5,888.0","5,858.0","5,982.0","5,834.0",491.41K,1.76%,2024,1,3
